# Assignment 3 #

We are given three columns of integers with a row for each node. The first two columns contain x and y coordinates of the node positions in a plane. The third column contains node costs. The goal is to select exactly 50% of the nodes (if the number of nodes is odd we round the number of nodes to be selected up) and form a Hamiltonian cycle (closed path) through this set of nodes such that the sum of the total length of the path plus the total cost of the selected nodes is minimized. The distances between nodes are calculated as Euclidean distances rounded mathematically to integer values. The distance matrix should be calculated just after reading an instance and then only the distance matrix (no nodes coordinates) should be accessed by optimization methods to allow instances defined only by distance matrices. 

## Read the data ##

In [80]:
import pandas as pd
import numpy as np
from numpy.typing import ArrayLike, NDArray
import matplotlib.pyplot as plt
import random
from tqdm import tqdm
from math import sqrt

In [81]:
# read data into dataframes
instances = {
    "A": pd.read_csv("data/TSPA.csv", sep=';', header=None, names=["x", "y", "cost"]),
    "B": pd.read_csv("data/TSPB.csv", sep=';', header=None, names=["x", "y", "cost"]),
    "C": pd.read_csv("data/TSPC.csv", sep=';', header=None, names=["x", "y", "cost"]),
    "D": pd.read_csv("data/TSPD.csv", sep=';', header=None, names=["x", "y", "cost"]),
}

In [84]:
def calculate_distance_matrix(df: pd.DataFrame) -> NDArray[np.int32]:
    """
    Calculate the distance matrix from the dataframe.
    The dataframe contains 'x' and 'y' columns for the coordinates.
    The distances are Euclidean, rounded to the nearest integer + the cost of the destination node.
    """
    coordinates = df[['x', 'y']].to_numpy()
    dist_matrix = np.zeros(shape=(len(df), len(df)))
    for i in range(len(coordinates)):
        for j in range(len(coordinates)):
            dist_matrix[i, j] = round(sqrt((coordinates[i, 0] - coordinates[j, 0])**2 + (coordinates[i, 1] - coordinates[j, 1])**2))
    return dist_matrix

In [85]:
distances_matrices = {
    "A": calculate_distance_matrix(instances["A"]),
    "B": calculate_distance_matrix(instances["B"]),
    "C": calculate_distance_matrix(instances["C"]),
    "D": calculate_distance_matrix(instances["D"])
}

costs = {
    "A": instances["A"]["cost"].to_numpy(),
    "B": instances["B"]["cost"].to_numpy(),
    "C": instances["C"]["cost"].to_numpy(),
    "D": instances["D"]["cost"].to_numpy()
}

In [5]:
def visualize_selected_route(
    selected_nodes_indices: ArrayLike, 
    dataframe: pd.DataFrame,
    title: str) -> None:
    """
    Visualize the selected route returned by the algorithm, including the cost of each node represented by a colormap.

    Parameters:
    selected_nodes_indices (list): Indices of the selected nodes in the route.
    dataframe (DataFrame): DataFrame containing 'x', 'y', and 'cost' columns for each node.
    """
    x = dataframe.loc[selected_nodes_indices, 'x']
    y = dataframe.loc[selected_nodes_indices, 'y']
    costs = dataframe.loc[selected_nodes_indices, 'cost']

    cmap = plt.cm.get_cmap('viridis')
    norm = plt.Normalize(vmin=min(costs), vmax=max(costs))

    plt.figure(figsize=(15, 10))
    scatter = plt.scatter(x, y, c=costs, cmap=cmap, norm=norm, s=100)
    plt.colorbar(scatter, label='Node Cost')

    for i, node in enumerate(selected_nodes_indices):
        start_node = selected_nodes_indices[i]
        end_node = selected_nodes_indices[(i + 1) % len(selected_nodes_indices)]
        plt.plot([x[start_node], x[end_node]], [y[start_node], y[end_node]], 'k-', lw=1)

    plt.title(title, fontsize=18)
    plt.xlabel('X Coordinate', fontsize=14)
    plt.ylabel('Y Coordinate', fontsize=14)
    plt.grid(True)
    plt.show()

In [86]:
def objective_function(solution: list[int], dist_matrix: list[list[int]], costs: list[int]) -> int:
    total_score = 0
    n = len(solution)
    for x in range(n):
        total_score += dist_matrix[solution[x - 1]][solution[x]]
        total_score += costs[solution[x]]
    return total_score

In [99]:
def two_edges_exchange(current_solution: list[int], 
                       current_distance: float, 
                       distance_matrix: list[list[int]]):
    """
    Generate new solutions by exchanging two edges in the current solution.

    :param current_solution: List of nodes in the current solution.
    :param current_score: The score of the current solution.
    :param distance_matrix: 2D list representing the distances between nodes.
    :return: A list of tuples where each tuple contains a new solution and its score.
    """
    n = len(current_solution)
    new_solutions = []

    for i in range(n - 2):
        for j in range(i + 2, n):
            # Create a new solution by reversing the order of nodes between i and j
            new_solution = current_solution[:i + 1] + current_solution[i + 1:j + 1][::-1] + current_solution[j + 1:]

            # Update the score for the new solution
            # Subtract the distances for the removed edges and add the distances for the new edges
            # print(f"Removed edges: ({current_solution[i]}->{current_solution[i+1]}) and ({current_solution[j]}->{current_solution[(j+1) % 2]})")
            # print(f"Added edges: ({current_solution[i]}->{current_solution[j]}) and ({current_solution[i+1]}->{current_solution[(j+1) % n]})")
            score_delta = (
                -distance_matrix[current_solution[i]][current_solution[i + 1]]
                -distance_matrix[current_solution[j]][current_solution[(j + 1) % n]]
                +distance_matrix[current_solution[i]][current_solution[j]]
                +distance_matrix[current_solution[i + 1]][current_solution[(j + 1) % n]]
            )
            new_score = current_distance + score_delta

            new_solutions.append((new_solution, 
                                  new_score, 
                                  objective_function(new_solution, distance_matrix, costs['A']),
                                  f"Removed edges: ({current_solution[i]}->{current_solution[i+1]}) and ({current_solution[j]}->{current_solution[(j+1) % 2]})",
                                  f"Added edges: ({current_solution[i]}->{current_solution[j]}) and ({current_solution[i+1]}->{current_solution[(j+1) % n]})"))
    
    return new_solutions

In [8]:
def two_nodes_exchange(selected, score, dist_matrix):
    solutions=[]
    for x in range(len(selected)): 
        for y in range(x+1,len(selected)):
            temp=selected[:]
            temp_score=score                  
            #usunięcie i zmienienie edgy poprzedzających node
            temp_score-=dist_matrix[temp[x-1]][temp[x]]
            temp_score+=dist_matrix[temp[x-1]][temp[y]]
            temp_score-=dist_matrix[temp[y-1]][temp[y]]
            temp_score+=dist_matrix[temp[y-1]][temp[x]]


            #kiedy byłby index x+1 out of range
            if temp[-1]==temp[x]:
                idx=temp[0]
            else: 
                idx=temp[x+1]

            #kiedy byłby index y+1 out of range 
            if temp[-1]==temp[y]:
                idy=temp[0]
            else:
                idy=temp[y+1]
            
            #usunięcie i zmienienie edgy następujących po node
            temp_score-=dist_matrix[temp[x]][idx]
            temp_score-=dist_matrix[temp[y]][idy]
            temp_score+=dist_matrix[temp[x]][idy]
            temp_score+=dist_matrix[temp[y]][idx]
    
            #podmiana nodów
            temp[x],temp[y]=temp[y],temp[x]


            solutions.append((temp,temp_score))
    return solutions

In [91]:
def inter_route_exchange(selected, unselected, score, distance_matrix):
    new_solutions = []
    for selected_node in selected:
        for new_node in unselected:
            new_solution = selected.copy()
            replaced_node_index = selected.index(selected_node)
            new_node_index = new_node - 1
            new_solution[replaced_node_index] = new_node

            prev_node_index = (replaced_node_index - 1) % len(selected)
            next_node_index = (replaced_node_index + 1) % len(selected)

            score_delta = (
                -distance_matrix[selected[prev_node_index] - 1][selected_node - 1]
                -distance_matrix[selected_node - 1][selected[next_node_index] - 1]
                +distance_matrix[selected[prev_node_index] - 1][new_node_index]
                +distance_matrix[new_node_index][selected[next_node_index] - 1]
            )
            new_score = score + score_delta

            new_solutions.append((new_solution, new_score))
    
    return new_solutions


In [93]:
start_solution = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
objective_function(start_solution, distances_matrices["A"], costs["A"])

22677.0

In [100]:
two_edges_exchange(start_solution, 22677.0, distances_matrices["A"])

[([0, 2, 1, 3, 4, 5, 6, 7, 8, 9],
  22676.0,
  22676.0,
  'Removed edges: (0->1) and (2->1)',
  'Added edges: (0->2) and (1->3)'),
 ([0, 3, 2, 1, 4, 5, 6, 7, 8, 9],
  21460.0,
  21460.0,
  'Removed edges: (0->1) and (3->0)',
  'Added edges: (0->3) and (1->4)'),
 ([0, 4, 3, 2, 1, 5, 6, 7, 8, 9],
  21596.0,
  21596.0,
  'Removed edges: (0->1) and (4->1)',
  'Added edges: (0->4) and (1->5)'),
 ([0, 5, 4, 3, 2, 1, 6, 7, 8, 9],
  24234.0,
  24234.0,
  'Removed edges: (0->1) and (5->0)',
  'Added edges: (0->5) and (1->6)'),
 ([0, 6, 5, 4, 3, 2, 1, 7, 8, 9],
  24073.0,
  24073.0,
  'Removed edges: (0->1) and (6->1)',
  'Added edges: (0->6) and (1->7)'),
 ([0, 7, 6, 5, 4, 3, 2, 1, 8, 9],
  20918.0,
  20918.0,
  'Removed edges: (0->1) and (7->0)',
  'Added edges: (0->7) and (1->8)'),
 ([0, 8, 7, 6, 5, 4, 3, 2, 1, 9],
  23169.0,
  23169.0,
  'Removed edges: (0->1) and (8->1)',
  'Added edges: (0->8) and (1->9)'),
 ([0, 9, 8, 7, 6, 5, 4, 3, 2, 1],
  22677.0,
  22677.0,
  'Removed edges: (0->1) an

In [ ]:
# current
[1,2,3,4,5]
# unselected
[6,7,8,9,10]
# robimy takie:
[6,2,3,4,5]
[1,6,3,4,5]
...
[1,2,3,4,10]